# Q CLI Setup for Strands SDK Development

This notebook provides step-by-step instructions to set up the Q CLI environment for developing AI agents using the AWS Strands SDK in an AWS event environment. The setup includes package managers, MCP servers, and development rules for spec-driven development. We will use Spec driven vibe coding to create an agent

## Prerequisites
- AWS CLI configured with appropriate permissions
- Internet connection for downloading packages
- Shell access (bash/zsh)
---

## Step 1: Install UV Package Manager

UV is a fast Python package manager that we'll use to manage dependencies and run tools. This command downloads and installs UV, then reloads the shell to make it available in the current session.

In [ ]:
curl -LsSf https://astral.sh/uv/install.sh | sh && exec $SHELL

## Step 2: Configure Strands MCP Server

This step sets up the Model Context Protocol (MCP) configuration for the Strands Agents server. The configuration:

- Creates the Q CLI configuration directory
- Configures the Strands MCP server to run via `uvx`
- Sets logging level to INFO for debugging
- Auto-approves safe documentation tools (`search_docs`, `fetch_doc`)

The MCP server provides access to Strands SDK documentation and examples directly within Q CLI.

In [ ]:
# Create the MCP configuration directory if it doesn't exist
mkdir -p ~/.aws/amazonq

# Add the Strands MCP server configuration to Q CLI
cat > ~/.aws/amazonq/mcp.json << 'EOF'
{
  "mcpServers": {
    "strands-agents": {
      "command": "uvx",
      "args": ["strands-agents-mcp-server"],
      "env": {
        "FASTMCP_LOG_LEVEL": "INFO"
      },
      "disabled": false,
      "autoApprove": [
        "search_docs",
        "fetch_doc"
      ]
    }
  }
}
EOF

## Step 3: Add AWS Documentation MCP Server

This command adds the AWS Documentation MCP server to Q CLI, providing access to official AWS service documentation. This is essential for understanding AWS services and their APIs while developing agents.

In [ ]:
q mcp add --name aws-documentation --command uvx --args aws-documentation-mcp-server

## Step 4: Add Amazon Bedrock AgentCore MCP Server

This step adds the Amazon Bedrock AgentCore MCP server, which provides tools for:

- Managing Bedrock agents and knowledge bases
- Querying knowledge bases with natural language
- Accessing AgentCore deployment capabilities

The `FASTMCP_LOG_LEVEL=ERROR` setting reduces log verbosity for cleaner output.

In [ ]:
q  mcp add --name amazon-bedrock-agentcore-mcp-server --command uvx --args "awslabs.amazon-bedrock-agentcore-mcp-server@latest" --env FASTMCP_LOG_LEVEL=ERROR

## Step 5: Download and Setup Development Rules

This step downloads the development rules file that defines:

- Project structure standards for Strands agents
- Development workflow guidelines
- Code generation constraints
- Testing requirements

The rules file is placed in the Q CLI rules directory to enable spec-driven development practices.

In [ ]:
wget https://github.com/Klosephi/spec-driven-strands/blob/3fe8b41b1867eddbdbfe219301186d8f5394c199/rule.md
mkdir -p .amazonq/rules
mv rule.md .amazonq/rules/

---

## Understanding Q CLI Rules and Spec-Driven Development

### What are Q CLI Rules?

**Q CLI Rules** are markdown files that define project-specific guidelines, coding standards, and behavioral rules for Amazon Q Developer CLI. These rules are automatically integrated into agent contexts, ensuring consistent behavior across your development workflow.

**Key Features:**
- **Workspace-specific**: Rules only apply to the current project directory
- **Automatic integration**: Default agent includes all rules from `.amazonq/rules/**/*.md`
- **Context loading**: Rules become part of the agent's context, influencing all interactions
- **Markdown format**: Standard markdown files with headers, code blocks, and examples

**Reference:** [AWS Documentation - Project Rules](https://docs.aws.amazon.com/amazonq/latest/qdeveloper-ug/command-line-project-rules.html)

### How Rules Work

1. **Storage**: Rules are stored in `.amazonq/rules/` directory as `.md` files
2. **Loading**: Q CLI automatically loads all rules when starting an agent session
3. **Context**: Rules become part of the agent's system context
4. **Influence**: Every code generation and suggestion follows the defined rules

### Rule Types

**Coding Standards:**
```markdown
# Python Coding Standards
- Use Black for code formatting
- Maximum line length: 88 characters
- Use type hints for all functions
```

**Architecture Guidelines:**
```markdown
# Architecture Guidelines
- Use microservices pattern for new features
- Implement circuit breaker for external APIs
- All services must expose health endpoints
```

**Security Policies:**
```markdown
# Security Guidelines
- All API endpoints require authentication
- Never log sensitive information
- Use parameterized queries
```

### What is Spec-Driven Development?

**Spec-Driven Development** is a methodology where development is guided by explicit specifications and rules that define:

- **Project Structure**: Standardized directory layouts and file organization
- **Development Workflow**: Step-by-step processes for building features
- **Code Generation**: Constraints and patterns for consistent code output
- **Testing Requirements**: Mandatory testing practices and coverage

### Benefits of Spec-Driven Development

1. **Consistency**: All team members follow the same patterns
2. **Quality**: Built-in best practices and standards
3. **Efficiency**: Reduced decision-making overhead
4. **Maintainability**: Predictable code structure and patterns
5. **Onboarding**: New developers can quickly understand project conventions

### How It Works with Strands SDK

The rules file we downloaded defines specific patterns for **AWS Strands Agent development**:

**Project Structure:**
```
project/
├── infra/           # Jupyter notebooks for AWS infrastructure
├── src/
│   ├── agents/      # Strands agent implementations
│   └── tools/       # Custom Python tools
└── tests/           # Unit and integration tests
```

**Development Workflow:**
1. **Research Phase**: Check PyPI for latest Strands SDK, validate existing tools
2. **Incremental Development**: Small, testable chunks with simple implementations
3. **Code Generation**: Precise scope, no assumptions about unstated requirements

**Strands Patterns**: The rules enforce proper use of Strands multi-agent patterns:
- **Graph**: Structured flowcharts with conditional logic
- **Swarm**: Dynamic collaborative teams with autonomous handoffs
- **Workflow**: Pre-defined task graphs (DAGs) for repeatable processes

**Reference:** [Strands Multi-Agent Patterns](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/multi-agent-patterns/)

### Strands SDK Multi-Agent Patterns

The Strands SDK provides three primary patterns for building multi-agent systems:

| Pattern | Use Case | Execution Flow | Best For |
|---------|----------|----------------|----------|
| **Graph** | Structured processes with conditional logic | Developer-defined flowchart with LLM decisions | Customer support routing, data validation |
| **Swarm** | Collaborative specialized agents | Autonomous handoffs between agents | Incident response, software development |
| **Workflow** | Repeatable complex processes | Pre-defined task dependencies (DAG) | Data pipelines, business processes |

**Key Principles:**
- **Orchestration**: Managing flow of information between agents
- **Specialization**: Each agent has specific role and tools
- **Collaboration**: Agents share information and build on each other's work

**Shared State**: Both Graph and Swarm support `invocation_state` for passing context:
```python
shared_state = {
    "user_id": "user123",
    "session_id": "sess456",
    "debug_mode": True
}

result = graph("Analyze data", invocation_state=shared_state)
```

### Model Context Protocol (MCP) Integration

**MCP** is an open protocol that standardizes how applications provide context to LLMs. The MCP servers we configured provide:

**Strands MCP Server:**
- Access to Strands SDK documentation
- Code examples and patterns
- Best practices for agent development

**AWS Documentation MCP Server:**
- Official AWS service documentation
- API references and guides
- Service integration patterns

**Bedrock AgentCore MCP Server:**
- Knowledge base management
- Natural language querying
- AgentCore deployment tools

These MCP servers work together with the rules to provide a comprehensive development environment where Q CLI has access to both the specifications (rules) and the documentation (MCP servers) needed for effective agent development.

---

## Next Steps

After completing this setup, you can proceed with agent development. Consider reviewing the following resources:

### Understanding AgentCore Deployment

To understand the current AgentCore deployment and project structure, refer to the bedrock-agentcore-workshop notebook:

**Reference:** `bedrock-agentcore-workshop/lab-05-frontend.ipynb`

This notebook provides insights into:
- AgentCore deployment patterns
- Frontend integration approaches
- Best practices for agent-to-frontend communication

### Project Objective: Visual Analytics Agent

**Goal:** Create a Visual Analytics Agent for Customer Support Memory Analysis

**Description:** Build a standalone Visual Analytics Agent that connects directly to the existing customer support memory system to generate real-time visual insights from conversation data.

**Requirements:**
- Deploy as independent service (no modifications to existing customer support agent)
- Connect directly to the same memory backend used by customer support agent
- Generate charts: conversation trends, resolution rates, topic distribution, session patterns

**Development Approach:**
1. Follow the rules defined in `.amazonq/rules/rule.md`
2. Use the Strands SDK patterns and tools
3. Implement incremental development with proper testing
4. Leverage the MCP servers for documentation and AWS service integration

**Recommended Pattern:** Consider using **Swarm** pattern for this project:
- **Data Analyst Agent**: Queries and processes conversation data
- **Visualization Agent**: Creates charts and visual representations
- **Insight Agent**: Generates analytical insights and recommendations
- **Report Agent**: Compiles final visual analytics reports

---

## Verification

To verify your setup is working correctly, you can run:

```bash
# Check Q CLI MCP servers
q mcp list

# Verify UV installation
uv --version

# Check if rules are in place
ls -la .amazonq/rules/

# Test Q CLI with rules
q chat "What are the project structure requirements for Strands agents?"
```

You should see:
- Configured MCP servers listed
- UV version information
- Rules file in the expected location
- Q CLI responding with information from the rules file

### Expected Output
When you ask Q CLI about project structure, it should reference the rules and provide information about the required directory layout:
- `infra/` for Jupyter notebooks
- `src/agents/` for Strands agent implementations
- `src/tools/` for custom Python tools
- `tests/` for unit and integration tests

---

## References

### AWS Documentation
- [Amazon Q Developer Project Rules](https://docs.aws.amazon.com/amazonq/latest/qdeveloper-ug/command-line-project-rules.html)
- [Amazon Q Developer Features](https://docs.aws.amazon.com/amazonq/latest/qdeveloper-ug/features.html)
- [AWS Prescriptive Guidance - Developer Workflows](https://docs.aws.amazon.com/prescriptive-guidance/latest/best-practices-code-generation/developer-workflows.html)

### Strands SDK Documentation
- [Strands Multi-Agent Patterns](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/multi-agent-patterns/)
- [Agent2Agent (A2A) Protocol](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agent-to-agent/)
- [CLI Reference Implementation](https://strandsagents.com/latest/documentation/docs/examples/python/cli-reference-agent/)

### Tools and Protocols
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)
- [UV Package Manager](https://docs.astral.sh/uv/)
- [Agent2Agent Protocol](https://a2aproject.github.io/A2A/latest/)